## Preprocessing Data

In [1]:
dir_ = 'C:/Users/Ronit/BTP/' # input only here

In [2]:
raw_data_dir = dir_+'data/'
processed_data_dir = dir_+'processed/'

In [3]:
import numpy as np
import pandas as pd
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from helper_func import get_memory_usage , size_of_fmt , reduce_dtypes , merge_by_concat

In [4]:
########################### Load Data #################################################################################
print('Main Data loaded')

raw_train_df = pd.read_csv(raw_data_dir+'sales_train_evaluation.csv')
raw_prices_df = pd.read_csv(raw_data_dir+'sell_prices.csv')
raw_calendar_df = pd.read_csv(raw_data_dir+'calendar.csv')

Main Data loaded


In [5]:
raw_train_df.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
5,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,0,0,1,0,0,5,2,0
6,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
7,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,7,0,6,0,15,5,4,1,40,32
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,1,0,0,0,0,0,0,0,1,0
9,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,0,0,1,0,2,1,1,0,0,1


##### 4. Reshaping / Tranforming Training set

In [6]:
TARGET = 'sales'         # Our main target
END_TRAIN = 1941         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [7]:
print('Reshaping raw_train_df')

# We can tranform horizontal representation to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
# and labels are 'd_' coulmns

index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(raw_train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

# We dont have much data and top of that we have too many features to we have to transrform the dataset to get most out of it
print('Train rows:', len(raw_train_df), len(grid_df)) # Now our training data has been increased

Reshaping raw_train_df
Train rows: 30490 59181090


In [8]:
raw_train_df.head()
print(raw_train_df.shape)

(30490, 1947)


In [9]:
# To be able to make predictions we need to add "test set" to our grid
add_grid = pd.DataFrame()
for i in range(1,29):
    temp_df = raw_train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)

# Removing ome temoprary DFs
del temp_df, add_grid

# We will not need original train_df anymore and can remove it
del raw_train_df

In [10]:
grid_df.shape

(60034810, 8)

In [11]:
grid_df.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
5,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
6,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
7,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12.0
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2.0
9,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0


In [12]:
# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',size_of_fmt(grid_df.memory_usage(index=True).sum())))

# We can free some memory 
# by converting "strings" to categorical
# it will not affect merging and 
# we will not lose any valuable data
for col in index_columns:
    grid_df[col] = grid_df[col].astype('category')

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',size_of_fmt(grid_df.memory_usage(index=True).sum())))

    Original grid_df:   3.6GiB
     Reduced grid_df:   1.3GiB


In [13]:
########################### Product Release date #################################################################################
print('Release week')

# It seems that leadings zero values in each train_df item row are not real 0 sales but mean absence for the item in the store
# we can save some memory by removing such zeros

# Prices are set by week so , we will have not very accurate release week 
release_df = raw_prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id','item_id','release']
release_df.head(10)

Release week


,store_id,item_id,release
0,CA_1,FOODS_1_001,11101
1,CA_1,FOODS_1_002,11101
2,CA_1,FOODS_1_003,11101
3,CA_1,FOODS_1_004,11206
4,CA_1,FOODS_1_005,11101
5,CA_1,FOODS_1_006,11101
6,CA_1,FOODS_1_008,11138
7,CA_1,FOODS_1_009,11206
8,CA_1,FOODS_1_010,11218
9,CA_1,FOODS_1_011,11101


In [14]:
# Now we can merge release_df
grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
del release_df

In [ ]:
# We want to remove some "zeros" rows from grid_df to do it we need wm_yr_wk column let's merge partly calendar_df to have it
grid_df = merge_by_concat(grid_df, raw_calendar_df[['wm_yr_wk','d']], ['d'])
                      
# Now we can cutoff some rows and safe memory 
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']].astype(np.int32)
grid_df = grid_df.reset_index(drop=True)

In [ ]:
# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# Should we keep release week as one of the features?  Only good CV can give the answer. Let's minify the release values.
# Min transformation will not help here as int16 -> Integer (-32768 to 32767)  and our grid_df['release'].max() serves for int16
# but we have have an idea how to transform other columns in case we will need it

grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release']

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

In [ ]:
########################### Save part 1 ####################################################################
print('Save Part 1')

# We have our BASE grid ready and can save it as pickle file for future use (model training)
grid_df.to_pickle(processed_data_dir+'grid_part_1.pkl')

print('Size:', grid_df.shape)